In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import pytz
import numpy as np
from capi import api_key

from models.gan_models import *
import yfinance as yf
import MetaTrader5 as mt5
import random
import os

In [21]:
#api_key = os.environ['COMET_API_KEY']
#api_key = ''

In [22]:
experiment = Experiment(
  api_key=api_key,
  project_name="tail-price",
  workspace="artaasd95"
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/artaasd95/tail-price/c0796ccd7b334eb2a0bd399521c9a8df
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Discriminator\Whole Train [11]  : (0.2500037994495658, 2903.2679585454075)
COMET INFO:     Main Generator\Whole Train [11] : (0.24998955896427466, 2909.2278901744785)
COMET INFO:     loss [202]                      : (0.006200707517564297, 3570.6318359375)
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (53.16 KB)
COMET INFO:     installed packages       : 1
COMET IN

In [21]:
mt5.initialize()

print('loading current tf data')
utc_from = datetime(2021, 1, 1, tzinfo=pytz.timezone("Asia/Nicosia"))
utc_to = datetime.now(pytz.timezone("Asia/Nicosia"))

data = mt5.copy_rates_range('XAUUSD', mt5.TIMEFRAME_H4, utc_from, utc_to)
data = pd.DataFrame(data)
time_data = data.time
data.drop(columns=['tick_volume', 'spread', 'real_volume'], inplace=True)

loading current tf data


In [23]:
data.to_csv('xau_2021_H4.csv')

In [4]:
data = pd.read_csv('xau_2021_H4.csv')

In [5]:
data

,Unnamed: 0,time,open,high,low,close
0,0,1609718400,1904.48,1918.60,1900.62,1916.57
1,1,1609732800,1916.65,1925.15,1915.44,1921.56
2,2,1609747200,1921.52,1935.09,1921.34,1932.03
3,3,1609761600,1931.95,1942.06,1927.79,1939.75
4,4,1609776000,1939.78,1944.33,1929.23,1936.92
...,...,...,...,...,...,...
5582,5582,1723780800,2458.38,2459.97,2450.72,2452.68
5583,5583,1723795200,2452.68,2464.50,2451.11,2462.32
5584,5584,1723809600,2462.32,2492.34,2461.17,2491.49
5585,5585,1723824000,2491.46,2500.08,2477.46,2495.82


In [6]:
input_size = 1
hidden_size = 512
seq_length = 50
num_layers = 8
batch_size = 64
num_epochs = 100
learning_rate = 0.01
lambda_gp = 10
n_critic = 5
# Initialize models
main_gen = MainGenerator(input_size, hidden_size, num_layers, 0.5, 1)
noise_gen = NoiseGenerator(input_size, 512, 1, 0.5)
discriminator = Discriminator(input_size, hidden_size, num_layers, 0.5)

In [7]:
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

In [15]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = torch.tensor(np.random.random((real_samples.size(0), 1)), dtype=torch.float)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = torch.ones(batch_size, 1)
    # Get gradient w.r.t. interpolates
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [9]:
main_gen.to(device)
noise_gen.to(device)
discriminator.to(device)

Discriminator(
  (lstm): LSTM(1, 512, num_layers=8, batch_first=True, dropout=0.5)
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.2)
)

In [10]:
optimizer_G = torch.optim.AdamW(list(main_gen.parameters()) + list(noise_gen.parameters()), lr=learning_rate)
optimizer_D = torch.optim.AdamW(discriminator.parameters(), lr=learning_rate)

optim_g_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_G, 0.1)
optim_d_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_D, 0.1)

In [11]:
cauchy_dist = torch.distributions.cauchy.Cauchy(loc=0, scale=0.5)

In [16]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    batch_start = 0
    g_loss_list = []
    noise_loss_list = []
    d_loss_list = []
    for idx in range(int(len(data)/batch_size)-1):
        price = torch.tensor(data[batch_start:batch_start+batch_size].close.values, dtype=torch.float).reshape(batch_size,1)
        #price = torch.tensor(random.sample(list(data[batch_start:batch_start+batch_size].close.values), batch_size), dtype=torch.float).reshape(batch_size,1)
        price = price.to(device)
        batch_start = batch_start + batch_size
        # Generate random noise inputs
        z1 = cauchy_dist.sample([batch_size, 1])
        z1 = z1.to(device)
        #z2 = cauchy_dist.sample([1])
        valid = torch.ones(batch_size, 1, device=device)
        fake = torch.zeros(batch_size, 1, device=device)
        # Generate fake data
        #fake_main = main_gen(price)
        fake_main = main_gen(z1)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise
        
        # Train disc
        optimizer_D.zero_grad()
        fake_main = main_gen(z1)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise

        # Real images
        real_validity = discriminator(price)
        # Fake images
        fake_validity = discriminator(fake_data)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, price, fake_data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        d_loss_list.append(d_loss.item())
        optimizer_D.step()

        optimizer_G.zero_grad()
        # Train the generator every n_critic steps
        if idx % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            fake_main = main_gen(z1)
            fake_noise = noise_gen(z1)
            fake_data = fake_main + fake_noise
            # Loss measures generator's ability to fool the discriminator
            # Train on fake data
            fake_validity = discriminator(fake_data)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            g_loss_list.append(g_loss.item())
            optimizer_G.step()


        

    optim_g_sched.step()
    optim_d_sched.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {np.average(d_loss_list)}  Generator Loss: {np.average(g_loss_list)}")
    experiment.log_metric('Main Generator\Whole Train', np.average(g_loss_list), step=epoch)
    experiment.log_metric('Discriminator\Whole Train', np.average(d_loss_list), step=epoch)


  0%|          | 0/100 [01:35<?, ?it/s]

Epoch [1/100]  Discriminator Loss: 4.9469789682432666  Generator Loss: 555.5506733734575


NameError: name 'experiment' is not defined

In [ ]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/artaasd95/tail-price/9f49e2d15e38475cabcfe319809b5eec
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Discriminator\Whole Train [50]  : (0.25955625224945156, 3416.939735186308)
COMET INFO:     Main Generator\Whole Train [50] : (0.26196861267089844, 3422.574053081056)
COMET INFO:     loss [860]                      : (0.2567949593067169, 10879.6005859375)
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (37.64 KB)
COMET INFO:     installed packages       : 1
COMET INFO

In [ ]:
torch.save(main_gen, 'checkpoints/main_gen_xau.pth')
torch.save(noise_gen, 'checkpoints/noise_gen_xau.pth')
torch.save(discriminator, 'checkpoints/discriminator_xau.pth')